# DESCARGA Y ALMACENAMIENTO DE DATOS EN LA NUBE

**Códigos para trabajar en entorno Google Colab, esto se hizo con el fin de no sobresaturar el equipo por el peso de los archivos de cada canal.
Se conectó todo a una cuenta drive y todos los archivos e imagenes quedaron almacenado en el directorio asignado.**

In [ ]:
# ============================================
# DESCARGA GOES-19 - ANÁLISIS SINÓPTICO
# Nevada en Atacama - Junio 2025
# Sudamérica - Horas Sinópticas (0,6,12,18 UTC)
# ============================================
#INSTALAMOS LAS LIBRERÍAS
!pip install boto3

import boto3
from datetime import datetime, timedelta
import os
from google.colab import drive, files
import zipfile
from botocore import UNSIGNED
from botocore.client import Config

In [ ]:
# ------------------------------------------------------------
# 1. MONTAR GOOGLE DRIVE
# ------------------------------------------------------------
drive.mount('/content/drive')

# ------------------------------------------------------------
# 2. CONFIGURACIÓN SIMPLE
# ------------------------------------------------------------
# Rango de fechas: 24 al 30 de Junio 2025
fecha_inicio = datetime(2025, 6, 24)
fecha_fin = datetime(2025, 6, 30)

# Horas sinópticas (0, 6, 12, 18 UTC)
horas = [0, 6, 10, 11 12, 18]

# Canales para análisis: C13 (IR), C08 (vapor), C09 (vapor), C10 (vapor)
canales = ["C13", "C08", "C09", "C10", "C11", "C12"]
bucket = "noaa-goes19"
producto = "ABI-L2-CMIPF"


Mounted at /content/drive


In [ ]:
# ------------------------------------------------------------
# 3. CARPETA EN DRIVE
# ------------------------------------------------------------
carpeta = f"/content/drive/MyDrive/GOES19_ATACAMA_{fecha_inicio.strftime('%Y%m%d')}_{fecha_fin.strftime('%Y%m%d')}"
os.makedirs(carpeta, exist_ok=True)
print(f"📁 Guardando en: {carpeta}")


📁 Guardando en: /content/drive/MyDrive/GOES19_ATACAMA_20250624_20250630


In [ ]:
# ------------------------------------------------------------
# 4. CONEXIÓN S3
# ------------------------------------------------------------
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))


In [ ]:
# ------------------------------------------------------------
# 5. DESCARGAR DÍA POR DÍA Y HORA POR HORA  ASIGNADA
# ------------------------------------------------------------
total = 0
fecha = fecha_inicio

while fecha <= fecha_fin:
    for hora in horas:
        # Crear ruta en el bucket
        doy = fecha.strftime('%j')
        year = fecha.strftime('%Y')
        prefix = f"{producto}/{year}/{doy}/{hora:02d}/"

        print(f"\n🔍 Buscando: {fecha.strftime('%Y-%m-%d')} {hora:02d}:00 UTC")

        try:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

            if 'Contents' in response:
                for obj in response['Contents']:
                    key = obj['Key']
                    # Filtrar solo canales que necesitamos
                    if any(c in key for c in canales):
                        filename = key.split('/')[-1]
                        # Crear subcarpeta por fecha
                        subcarpeta = os.path.join(carpeta, f"{fecha.strftime('%Y%m%d')}_{hora:02d}")
                        os.makedirs(subcarpeta, exist_ok=True)

                        # Descargar archivo
                        ruta_local = os.path.join(subcarpeta, filename)
                        if not os.path.exists(ruta_local):
                            s3.download_file(bucket, key, ruta_local)
                            tamaño = os.path.getsize(ruta_local)/1024/1024
                            print(f"  ✅ {filename} ({tamaño:.1f} MB)")
                            total += 1
                        else:
                            print(f"  ⏩ {filename} ya existe")
        except:
            print(f"  ⚠️ Sin datos para esta hora")

    fecha += timedelta(days=1)



🔍 Buscando: 2025-06-24 00:00 UTC
  ✅ OR_ABI-L2-CMIPF-M6C08_G19_s20251750000202_e20251750009510_c20251750009571.nc (17.4 MB)
  ✅ OR_ABI-L2-CMIPF-M6C08_G19_s20251750010202_e20251750019510_c20251750019581.nc (17.3 MB)
  ✅ OR_ABI-L2-CMIPF-M6C08_G19_s20251750020202_e20251750029510_c20251750029571.nc (17.3 MB)
  ✅ OR_ABI-L2-CMIPF-M6C08_G19_s20251750030202_e20251750039510_c20251750039567.nc (17.3 MB)
  ✅ OR_ABI-L2-CMIPF-M6C08_G19_s20251750040202_e20251750049510_c20251750049574.nc (17.3 MB)
  ✅ OR_ABI-L2-CMIPF-M6C08_G19_s20251750050202_e20251750059510_c20251750059585.nc (17.3 MB)
  ✅ OR_ABI-L2-CMIPF-M6C09_G19_s20251750000202_e20251750009516_c20251750009561.nc (16.6 MB)
  ✅ OR_ABI-L2-CMIPF-M6C09_G19_s20251750010202_e20251750019516_c20251750019570.nc (16.6 MB)
  ✅ OR_ABI-L2-CMIPF-M6C09_G19_s20251750020202_e20251750029516_c20251750029590.nc (16.6 MB)
  ✅ OR_ABI-L2-CMIPF-M6C09_G19_s20251750030202_e20251750039516_c20251750039563.nc (16.6 MB)
  ✅ OR_ABI-L2-CMIPF-M6C09_G19_s20251750040202_e202517500